In [2]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForMaskedLM, BertweetTokenizer

config = AutoConfig.from_pretrained('vinai/bertweet-base')
tokenizer = AutoTokenizer.from_pretrained('vinai/bertweet-base')
model = AutoModelForMaskedLM.from_config(config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from glob import glob
from datasets import Dataset

import re
import pandas as pd

from parse import mask_data_loading

data_url = '../crawler/stock/data/**.json'
url = glob(data_url)[-1]
data, symbols = mask_data_loading(url, tokenizer)

dataset = Dataset.from_pandas(data.loc[:, ['labels', 'sentense']])
dataset = dataset.remove_columns('__index_level_0__')

special_tokens_dict = {'additional_special_tokens': list(symbols)}
tokenizer.add_special_tokens(special_tokens_dict)
model.resize_token_embeddings(len(tokenizer))

Embedding(64614, 768)

In [4]:
tokenizer.save_pretrained('./symbol-vocab')

('./symbol-vocab\\tokenizer_config.json',
 './symbol-vocab\\special_tokens_map.json',
 './symbol-vocab\\vocab.txt',
 './symbol-vocab\\bpe.codes',
 './symbol-vocab\\added_tokens.json')

In [4]:
def encode(example):
    sentense = example['sentense']
    label = example['labels']

    label = tokenizer(label, padding='max_length', truncation=True)
    # 101, 51, 1234, 12541, 151
    result = tokenizer(sentense, padding='max_length', truncation=True)
    # 101, 103, 103, 103
    result['label_ids'] = label['input_ids']

    # masked_position = [i for i in range(len(result['input_ids'])) if result['input_ids'][i] == tokenizer.mask_token_id]
    # result['decoder_input_ids'] = label['input_ids']
    # result['labels'] = [-100 for i in label['input_ids']]
    # for i in range(len(result['labels'])):
    #     if not i in masked_position:
    #         result['labels'][i] = -100

    return result

encoded_dataset = dataset.map(encode, batched=True)

# print(encoded_dataset[0]['sentense'])
# print(encoded_dataset[0]['input_ids'])
# print(encoded_dataset[0]['labels'])
# print(encoded_dataset[0]['label_ids'])
# print(tokenizer.ids_to_tokens)
# print(tokenizer.decode(encoded_dataset[0]['label_ids']))

100%|██████████| 16/16 [00:08<00:00,  1.82ba/s]


In [5]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

# samples = encoded_dataset[:2]
# for chunk in data_collator(samples)["input_ids"]:
#     print(f"\n'>>> {tokenizer.decode(chunk)}'")

# Training

In [6]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
from transformers import TrainingArguments, Trainer, Seq2SeqTrainingArguments, Seq2SeqTrainer

# training_args = TrainingArguments(
#     output_dir="test_trainer",
#     per_device_train_batch_size=2,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=encoded_dataset,
#     compute_metrics=compute_metrics,
# )

# Seq2SeqTrainer
training_args = Seq2SeqTrainingArguments(
    output_dir="test_trainer",
    per_device_train_batch_size=4,
    num_train_epochs=50,
    save_steps=10000
)

train_datset = encoded_dataset.shuffle().select(range(4000))

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_datset,
    compute_metrics=compute_metrics,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: sentense. If sentense are not expected by `RobertaForMaskedLM.forward`,  you can safely ignore this message.
C:\Users\douli\anaconda3\envs\stock-nlp\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4000
  Num Epochs = 50
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 50000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB

  1%|          | 501/50000 [01:10<2:32:30,  5.41it/s]

{'loss': 2.2814, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.5}


  2%|▏         | 1001/50000 [02:17<2:29:30,  5.46it/s]

{'loss': 0.1828, 'learning_rate': 4.9e-05, 'epoch': 1.0}


...

100%|██████████| 50000/50000 [1:48:14<00:00,  4.92it/s]Saving model checkpoint to test_trainer\checkpoint-50000
Configuration saved in test_trainer\checkpoint-50000\config.json


{'loss': 0.0009, 'learning_rate': 0.0, 'epoch': 50.0}


Model weights saved in test_trainer\checkpoint-50000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 50000/50000 [1:48:24<00:00,  7.69it/s]

{'train_runtime': 6508.0232, 'train_samples_per_second': 30.731, 'train_steps_per_second': 7.683, 'train_loss': 0.03217216126203537, 'epoch': 50.0}


TrainOutput(global_step=50000, training_loss=0.03217216126203537, metrics={'train_runtime': 6508.0232, 'train_samples_per_second': 30.731, 'train_steps_per_second': 7.683, 'train_loss': 0.03217216126203537, 'epoch': 50.0})

In [9]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model=model,
    tokenizer=tokenizer,
    device=0
)

s = f'{tokenizer.mask_token} to the sky!!!'
fill_mask(s)

[{'score': 0.999998927116394,
  'token': 64110,
  'token_str': '$ T S L A',
  'sequence': 'to the sky!!!'},
 {'score': 5.828806237673234e-08,
  'token': 64082,
  'token_str': '$ M S F T',
  'sequence': 'to the sky!!!'},
 {'score': 3.953035232484581e-08,
  'token': 64174,
  'token_str': '$ A A P L',
  'sequence': 'to the sky!!!'},
 {'score': 3.7064779689899296e-08,
  'token': 3,
  'token_str': '< u n k >',
  'sequence': 'to the sky!!!'},
 {'score': 3.131322046101559e-08,
  'token': 64560,
  'token_str': '$ P L T R',
  'sequence': 'to the sky!!!'}]

In [ ]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary())